In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

In [ ]:
path_data

In [ ]:
os.path.exists(scr)

In [ ]:
/home/demitau/crnl_workstation/data_Quentin/full_experiments/data2/df_all_multi_tsz_.pkl.zip

In [ ]:
bdf = pjoin(path_data, 'df_all_multi_tsz_.pkl.zip')

In [ ]:
consist = checkTgtConsistency(behav_df_cur,epochs)
print(consist)

In [ ]:
dfev.columns['']

In [ ]:
behav_df_cur['trigger_present'].all()

In [ ]:
behav_df_cur['target_codes']

In [ ]:
#behav_df_cur['target_inds'] 

In [ ]:
dfev['target_code']

In [ ]:
behav_df_cur['target_codes']

In [ ]:
dfev.columns

In [ ]:
# read full df
# add index valid column

In [ ]:
(dfev['code_x'] == dfev['code_y']).all()
(dfev['type_x'] == dfev['type_y']).all()

In [ ]:
dfev.columns

In [ ]:
behav_df_cur.trial_index.values

In [ ]:
# only FEEDBACK_PHASE. 
# To get acces to other phases: 
# for REST one has to subtract fixed value from the first
behav_df_cur

# two datastes -- for feedback (trigger_nsample is the sample index on which feedback starts ) 
# and for target (trigger_nsample is the sample index on which target appears )
# normally only  phase, trigger, error_distance

In [ ]:
for coln in behav_df_cur.columns:
    print (coln,' -- ')

In [ ]:
#behav_df_cur['trial_type']

In [ ]:
args.time_locked

In [ ]:
col_to_drop = ['phase','environment','trials','subject',
               'time_diff_phase','time_since_last', 'target_codes',
'phase_duration',  
'perturbation_',
'error_type',
'non_hit_shifted',
'correction',
'belief_',
'belief',
'trial_inds_glob_prevlike_error',
'trial_inds_glob_nextlike_error',
'prev_err_sens',
'retention_factor_s',  
'vals_for_corr1',
'vals_for_corr2',
'dist_rad_from_prevtgt2',  
'next_error',  
'next_belief',  
'hit_mestd1',   
'err_sens_trunc',  
'movement','trigger']   


In [ ]:
op.join(path_data_tmp_cursubj, fn_flt_raw )

In [ ]:
args.behav_file

In [ ]:
/home/demitau/crnl_workstation/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/behavdata/behav_visuomotor_df_upd_perframe.pkl.zip

In [ ]:
import shutil
#shutil.copy(fn_events_full,pjoin(exp_dir, f'VisuoMotor_no_filter_with_ICA_eve.txt'))
fn_events_full

In [ ]:
dfev_full.query('phase.isnull()').value_counts('code')

In [ ]:
dfev_full.query('phase.isnull()').code.unique()

In [ ]:
#dfpf[['time','phase']]

In [ ]:
dfs_beh__[0]['time_locked'],dfs_beh__[1]['time_locked']

In [ ]:
dfs_beh__[0]['time'],dfs_beh__[1]['time']

In [ ]:
exp_dir = pjoin(os.path.dirname(path_data), '_export_Saxena') 

scr = pjoin(scripts_dir,'dec_err_sens_sliding2_preopt.py')
dfs__ = []
dfs_beh__ = []
for tl__ in ['target','feedback']:
#for tl__ in ['feedback']:
    ipy.run_line_magic('run',f'  -i {scr} --subject {subjects[0]} --exit_after enforce_consist '
                       '--param_file /home/demitau/memerr_code/params/dec_err_sens_sliding.ini --slide_window_shift 0.2'
                      f' --behav_file {bdf} --retention_factor 1. --time_locked {tl__} --use_preloaded_flt_raw 1')
    
    dfs__ += [dfev]
    dfs_beh__ += [behav_df_cur]
        
    exp_fname = pjoin(exp_dir, f'{tl__}_dfev.parquet')    
    dfev.to_parquet(exp_fname)
    
exp_fname = pjoin(exp_dir, f'feedback.parquet')
behav_df_cur_sm = behav_df_cur.drop(columns = col_to_drop)
behav_df_cur_sm.to_parquet(exp_fname)    

assert set(dfs__[0].trial_index.unique()) == set(dfs__[1].trial_index.unique()) 
    
dfpf = pd.read_pickle(pjoin(path_data,subject,'behavdata','behav_visuomotor_df_upd_perframe.pkl.zip'))
#dfpf = dfpf.rename(columns={'trials':'trial_index'})
valid_trials = dfev.trial_index.unique()
dfpf['triggers_found'] = dfpf.trials.isin( valid_trials )
exp_fname = pjoin(exp_dir, f'df_perframe.parquet')  
dfpf.to_parquet(exp_fname)
    
#np.savez( pjoin(exp_dir, f'events.npz') ,events)
from meg_proc import events2df_NIH
from config2 import trigger2phase
dfev_full = events2df_NIH(events, advcols=False)
dfev_full['phase'] = dfev_full['code'].apply(lambda x: trigger2phase.get(x,None))
dfev_full.loc[dfev_full.type == 'target','phase'] = 'TARGET_PHASE'
dfev_full.loc[dfev_full.type == 'feedback','phase'] = 'FEEDBACK_PHASE'

exp_fname = pjoin(exp_dir, f'dfev_full.parquet')    
dfev_full.to_parquet(exp_fname)

In [ ]:
time_locked

In [ ]:
#epochs

In [ ]:
dfev['target_code']

In [ ]:
consist

In [ ]:
time_locked

In [ ]:
behav_df_full[]

In [ ]:
behav_df_full['retention_factor_s']

In [ ]:
behav_df_full.columns

In [ ]:
behav_fnf

In [ ]:
par['colns_ES']